In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('../data/melb_data.csv')

# Separate target from predictors
y = data.Price
data = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(data, 
                                               y, train_size=0.8, test_size=0.2, random_state=0)

In [85]:
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [col for col in X_train_full.columns 
                        if X_train_full[col].nunique() < 10 and 
                        X_train_full[col].dtype == 'object']

numerical_cols = [col for col in X_train_full.columns if X_train_full[col].dtype 
                  in ['int64', 'float64']]

In [86]:
print(categorical_cols)
print(numerical_cols)

['Type', 'Method', 'Regionname']
['Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount']


In [87]:
# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [88]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


### Three step pipeline

**Step 1** Use the `ColumnTransformer` class to bundle together different preprocessing steps:
    * Impute missing values in *numerical* data
    * Impute missing values and apply one-hot encoding to *categorical* data

In [108]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data

"""
If “constant”, then replace missing values with fill_value. 
Can be used with strings or numeric data.

When strategy == “constant”, fill_value is used to replace all occurrences of missing_values. 
If left to the default, fill_value will be 0 when imputing numerical data and “missing_value” 
for strings or object data types.
"""
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
                          ('imputer', SimpleImputer(strategy='most_frequent')), 
                          ('onehot', OneHotEncoder(handle_unknown='ignore'))
                            ])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
                transformers=[
                ('num', numerical_transformer, numerical_cols),
                ('cat', categorical_transformer, categorical_cols)
                ])

**Step 2** Define the model

We use the `RandomForestRegressor` class

In [109]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

**Step 3** Create and evaluate the pipeline
Use the `Pipeline` class to define a pipeline that bundles the preprocessing and modeling steps. 

This way we:
* Preprocess the training data and fit the model in a single line of code!
* Supply the (unprocessed) features in `X_valid` to the `predict()` command, and the pipeline automatically preprocesses those features then generates predictions. 

In [110]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Preprocessing of training data and fit model
pipeline.fit(X_train, y_train)

# Preprocessing of validation data and get predictions
preds = pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


In [106]:
[col for col in X_train_full.columns if X_train_full[col].isnull().any()]

['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']